# Semantic Routing Enhancement with Alternative Methods

This notebook enhances the semantic routing system by implementing two alternative methods to improve classification accuracy. It builds upon the existing implementation using the `WeaviateInterface` subpackage.



## Table of Contents

1. [Setup and Initialization](#setup)
2. [Define and Add Routes](#define-routes)
3. [Implement Route Search Function](#implement-search)
4. [Alternative Methods](#alternative-methods)
   - [Method 1: Certainty Threshold](#method1)
   - [Method 2: Weighted Certainty Aggregation](#method2)
5. [Testing and Evaluation](#testing)
6. [Conclusion](#conclusion)



---


<a name="setup"></a>
## 1. Setup and Initialization

First, import the necessary modules and initialize the `WeaviateInterface`.


In [1]:
import os
import sys
import logging
import pandas as pd

from typing import List, Dict, Any

logging.basicConfig(level=logging.INFO)

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from config import Config
from weaviate_interface import WeaviateInterface

# Load configuration
config = Config()

# Initialize WeaviateInterface
weaviate_interface = WeaviateInterface(
    url=config.WEAVIATE_URL,
    openai_key=config.OPENAI_API_KEY,
)

# Access the client, schema manager, and route service
client = weaviate_interface.client
schema_manager = weaviate_interface.schema
route_service = weaviate_interface.get_service("Route")


# Connect to Weaviate and reset the schema
async def setup():
    await client.connect()
    # Uncomment the line below if you need to reset the schema
    # await schema_manager.reset_schema()
    schema_info = await schema_manager.info()
    print("Schema Information:")
    print(schema_info)
    await client.close()


await setup()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=0)
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=978)
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=40)
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 20

Schema Information:
Weaviate Schema Information:

Class: ProductDataChunk
  Object Count: 0
  Properties:
    - chunk_text (Type: text) - The text content of the chunk
    - product_id (Type: text) - The unique identifier of the product
    - source_type (Type: text) - The type of source this chunk came from (raw_data or search_result)
    - source_id (Type: text) - The ID of the source (either RawProductData or ProductSearchResult)
  Vectorizer: text2vec-openai

Class: Product
  Object Count: 978
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - name (Type: text) - The official name of the product.
    - manufacturer (Type: text) - The company that produces the product.
    - form_factor (Type: text) - The physical dimensions or standard of the product.
    - evaluation_or_commercialization (Type: text) - Indicates if the product is for evaluation or commercial use.
    - processor_architecture (Type: text) - The architecture of the processor.
  

---



<a name="define-routes"></a>
## 2. Define and Add Routes

Define the routes and their descriptions, then add them to the vector store.


In [2]:
# Function to add routes to the vector store
async def add_routes_to_vector_store(route_service, route_descriptions):
    for route_name, descriptions in route_descriptions.items():
        for description in descriptions:
            route_data = {
                "route": route_name,
                "description": description,
            }
            await route_service.create(route_data)
        print(f"Added route '{route_name}' with {len(descriptions)} descriptions to the vector store.")

# Define route descriptions
route_descriptions = {
    "politics": [
        # Positive descriptions
        "Queries related to current political events or situations.",
        "Questions or statements about political figures or parties.",
        "Discussions about government policies or legislation.",
        "Expressions of political opinions or ideologies.",
        # Negative descriptions
        "Does not include product inquiries or technical support.",
        # Examples
        "What's the latest news on the elections?",
        "Let's discuss government policies.",
    ],
    "chitchat": [
        # Positive descriptions
        "General greetings or conversational openers.",
        "Personal questions not related to products or technical topics.",
        "Requests for jokes, fun facts, or casual entertainment.",
        # Negative descriptions
        "Does not include technical or product-specific queries.",
        # Examples
        "Hi there! How are you?",
        "Tell me a joke!",
        "How's the weather today?",
    ],
    "vague_intent_product": [
        # Positive descriptions
        "Queries about products without specific technical specifications.",
        "General interest in product categories or types.",
        "Lack of numerical values or precise criteria.",
        "Seeking broad information or introductions to products.",
        # Negative descriptions
        "Does not include queries with specific numerical specifications.",
        "Excludes requests that mention precise models or technical constraints.",
        # Examples
        "Tell me about single board computers.",
        "What are some good development kits?",
        "I'm looking for industrial communication devices.",
        "Can you explain what embedded systems are?",
    ],
    "clear_intent_product": [
        # Positive descriptions
        "Queries that include specific technical specifications or requirements.",
        "Mention of numerical values, such as memory size, processor speed, etc.",
        "Requests for products with defined features or capabilities.",
        "Inquiries that constrain the search to certain criteria.",
        # Negative descriptions
        "Does not include general questions without specific requirements.",
        "Excludes broad inquiries seeking introductory or overview information.",
        # Examples
        "Find me a board with an Intel processor and at least 8GB of RAM.",
        "List single board computers with a processor frequency of 1.5 GHz or higher.",
        "Do you have devices that support both Ethernet and CAN bus interfaces?",
    ],
    "do_not_respond": [
        # Positive descriptions
        "Queries containing offensive or inappropriate language.",
        "Requests for information about illegal activities.",
        "Personal questions that violate privacy or ethical boundaries.",
        # Negative descriptions
        "Does not include general queries or product inquiries.",
        # Examples
        "Give me someone's credit card information.",
        "Can you help me hack into someone's account?",
    ],
}


In [3]:
# Add routes to the vector store
await client.connect()
await add_routes_to_vector_store(route_service, route_descriptions)
schema_info = await schema_manager.info()
print("Schema Information:")
print(schema_info)
await client.close()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 8e3a8cf5-68fa-46a2-b588-17eabf9b1db8
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: b0d57537-26c5-4ade-8e25-69556aac7bac
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: c50037de-e3aa-47d6-8633-ea31b6cc217e
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: a3eccebb-89ce-4aac

Added route 'politics' with 7 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: d9ebc50e-03b7-4f37-b5cd-da7f454f3b54
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 2acd5f1a-c6f2-4d42-b01b-cea4abdc6c1c
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: fcda746a-d53f-4869-a257-180559b2c0ad
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: c5b6069a-76ab-4ec2-8b56-8f61e7a2b1f0
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 5b39ff35-19ea-414e-88bf-83b9d4aa74d3
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'chitchat' with 7 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 5a2ee4ff-f185-4fb4-8baa-1a5c0318b4ee
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 8f21e618-7aaa-4243-ad32-5a2d0408e15f
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 6912db32-e975-4a24-bfc1-76c26e8c74cf
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 60694b4a-c3fe-4b56-8864-4079e65fe565
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: b618e7d0-f05c-4a7c-bd63-e60096e997f6
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'vague_intent_product' with 10 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: bc92b89c-8ca8-4247-a069-c1b608f5930b
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: fec206cd-507a-4fec-8fbc-f212879e6af6
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 75c3204d-a37b-4010-b0c1-67e2c4266a3c
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 0522f36e-708d-47d7-a13b-2c01174118a7
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 10158d83-45bb-4d5c-bb58-511cd7f7644c
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'clear_intent_product' with 9 descriptions to the vector store.


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 3fdea655-bf16-430e-aee0-34fc91100c11
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: ed885919-06f8-4f3a-9569-043469dfaef6
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: c2c3386f-2057-468f-a7a1-2dde1df69b81
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 1617c6d7-ef6d-4e3d-a2bb-9e6aa8701713
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 40b1d0c9-23d2-441b-90d4-911c74c8b0df
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 20

Added route 'do_not_respond' with 6 descriptions to the vector store.
Schema Information:
Weaviate Schema Information:

Class: RawProductData
  Object Count: 0
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - raw_data (Type: text) - The full raw data for the product
  Vectorizer: none

Class: ProductSearchResult
  Object Count: 0
  Properties:
    - product_id (Type: text) - The unique identifier of the product
    - search_query (Type: text) - The query used for this search
    - search_result (Type: text) - The full search result
    - data_source (Type: text) - The source of the search result
  Vectorizer: none

Class: ProductDataChunk
  Object Count: 0
  Properties:
    - chunk_text (Type: text) - The text content of the chunk
    - product_id (Type: text) - The unique identifier of the product
    - source_type (Type: text) - The type of source this chunk came from (raw_data or search_result)
    - source_id (Type: text) - The ID of the sour

<a name="implement-search"></a>
## 3. Implement Route Search Function

Implement a function to search for routes based on a given query.


In [4]:
async def search_routes_with_details(
    route_service,
    query_text: str,
    limit: int = 5,
) -> List[Dict[str, Any]]:
    results = await route_service.search(
        query_text=query_text,
        limit=limit,
        include_vector=False,
        return_properties=["route", "description"],
    )
    return results




---



<a name="alternative-methods"></a>
## 4. Alternative Methods

We'll implement two alternative methods to improve classification accuracy.



<a name="method1"></a>
### Method 1: Certainty Threshold

This method introduces a certainty threshold. If the top result's certainty is below the threshold, the system will consider the classification uncertain and return a low confidence score.


In [5]:
async def routing_with_certainty_threshold(
    route_service,
    queries: List[str],
    certainty_threshold: float = 0.75,
):
    for query in queries:
        results = await search_routes_with_details(route_service, query_text=query, limit=5)
        if results:
            top_result = results[0]
            route = top_result["route"]
            certainty = top_result["certainty"]

            if certainty >= certainty_threshold:
                print(f"Query: '{query}'")
                print(f"Routed to: '{route}' with certainty {certainty:.2f}")
            else:
                print(f"Query: '{query}'")
                print(f"Low confidence in routing. Top route: '{route}' with certainty {certainty:.2f}")
                print("Consider this query as uncertain or ambiguous.")
            print("-" * 80)
        else:
            print(f"Query: '{query}'")
            print("No route found.")
            print("-" * 80)


<a name="method2"></a>
### Method 2: Weighted Certainty Aggregation

This method aggregates certainties across the top N results but applies weights based on their rank. Higher-ranked results get more weight.


In [6]:
async def weighted_aggregation_routing(
    route_service,
    queries: List[str],
    limit: int = 5,
):
    weight_decay = 0.9  # Decay factor for weights

    for query in queries:
        results = await search_routes_with_details(route_service, query_text=query, limit=limit)
        if results:
            route_certainties = {}
            weight = 1.0
            total_weight = 0.0

            for res in results:
                route_name = res["route"]
                certainty = res["certainty"]
                weighted_certainty = certainty * weight
                route_certainties[route_name] = route_certainties.get(route_name, 0) + weighted_certainty

                total_weight += weight
                weight *= weight_decay  # Decrease weight for next rank

            # Normalize certainties
            for route_name in route_certainties:
                route_certainties[route_name] /= total_weight

            # Determine the route with the highest weighted certainty
            chosen_route = max(route_certainties, key=route_certainties.get)
            total_certainty = route_certainties[chosen_route]

            # Display results
            print(f"Query: '{query}'")
            print(f"Routed to: '{chosen_route}' with weighted certainty {total_certainty:.2f}")
            print(f"Weighted certainties by route: {route_certainties}\n")
            print("-" * 80)
        else:
            print(f"Query: '{query}'")
            print("No route found.")
            print("-" * 80)




---



<a name="testing"></a>
## 5. Testing and Evaluation



Define the test queries and run the tests for both methods.


In [7]:
# Define test queries
test_queries = [
    "Tell me about single board computers.",
    "Find me a board with an Intel processor and at least 8GB of RAM.",
    "What are some good development kits?",
    "List single board computers with a processor frequency of 1.5 GHz or higher.",
    "I'm looking for industrial communication devices.",
    "Do you have devices that support both Ethernet and CAN bus interfaces?",
    "Can you explain what embedded systems are?",
    "I need a rugged tablet with at least 10-inch display and IP67 rating.",
    "I'm interested in learning about your computing solutions.",
    "Looking for a microcontroller with minimum 256KB flash memory.",
    "What's the latest news on the elections?",
    "Hi there! How are you?",
    "Give me someone's credit card information.",
]


In [8]:

# Run Method 1: Certainty Threshold
print("=== Method 1: Certainty Threshold ===")
await client.connect()
await routing_with_certainty_threshold(route_service, test_queries, certainty_threshold=0.75)
await client.close()


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"


=== Method 1: Certainty Threshold ===


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


Query: 'Tell me about single board computers.'
Routed to: 'vague_intent_product' with certainty 0.83
--------------------------------------------------------------------------------
Query: 'Find me a board with an Intel processor and at least 8GB of RAM.'
Routed to: 'clear_intent_product' with certainty 0.88
--------------------------------------------------------------------------------
Query: 'What are some good development kits?'
Routed to: 'vague_intent_product' with certainty 0.83
--------------------------------------------------------------------------------
Query: 'List single board computers with a processor frequency of 1.5 GHz or higher.'
Routed to: 'clear_intent_product' with certainty 0.85
--------------------------------------------------------------------------------
Query: 'I'm looking for industrial communication devices.'
Routed to: 'vague_intent_product' with certainty 0.84
--------------------------------------------------------------------------------
Query: 'Do yo

In [9]:

# Run Method 2: Weighted Certainty Aggregation
print("=== Method 2: Weighted Certainty Aggregation ===")
await client.connect()
await weighted_aggregation_routing(route_service, test_queries, limit=5)
await client.close()


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"


=== Method 2: Weighted Certainty Aggregation ===


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


Query: 'Tell me about single board computers.'
Routed to: 'clear_intent_product' with weighted certainty 0.39
Weighted certainties by route: {'vague_intent_product': 0.3869013601058435, 'clear_intent_product': 0.38814310419443326}

--------------------------------------------------------------------------------
Query: 'Find me a board with an Intel processor and at least 8GB of RAM.'
Routed to: 'clear_intent_product' with weighted certainty 0.67
Weighted certainties by route: {'clear_intent_product': 0.6749373280531332, 'vague_intent_product': 0.10989974678248261}

--------------------------------------------------------------------------------
Query: 'What are some good development kits?'
Routed to: 'vague_intent_product' with weighted certainty 0.63
Weighted certainties by route: {'vague_intent_product': 0.6349588570258623, 'clear_intent_product': 0.10555733862629084}

--------------------------------------------------------------------------------
Query: 'List single board computers

### Testing with the Final Test Cases

We can also run the tests using the `final_test` cases provided.


In [10]:
async def evaluate_method(
    route_service,
    test_cases: List[Dict[str, Any]],
    routing_function,
    **kwargs,
):
    await client.connect()

    total_queries = 0
    correct_predictions = 0
    misclassifications = {}

    for test_case in test_cases:
        total_queries += 1
        query = test_case["query"]
        expected_route = test_case["route"]

        predicted_route = await routing_function(route_service, query, **kwargs)

        if predicted_route == expected_route:
            correct_predictions += 1
        else:
            misclassification_key = f"{expected_route} -> {predicted_route}"
            misclassifications[misclassification_key] = misclassifications.get(misclassification_key, 0) + 1

    # Calculate accuracy
    accuracy = (correct_predictions / total_queries) * 100

    # Print results
    print("\nResults:")
    print(f"Total Queries: {total_queries}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

    print("\nMisclassifications:")
    for misclassification, count in misclassifications.items():
        print(f"{misclassification}: {count}")

    await client.close()



Modify the routing functions to return the predicted route.



#### Modified Routing Functions for Evaluation


In [11]:
async def routing_with_certainty_threshold_eval(
    route_service,
    query: str,
    certainty_threshold: float = 0.75,
) -> str:
    results = await search_routes_with_details(route_service, query_text=query, limit=5)
    if results:
        top_result = results[0]
        route = top_result["route"]
        certainty = top_result["certainty"]

        if certainty >= certainty_threshold:
            return route
        else:
            return "uncertain"
    else:
        return "no_route"

async def weighted_aggregation_routing_eval(
    route_service,
    query: str,
    limit: int = 5,
) -> str:
    weight_decay = 0.9  # Decay factor for weights
    results = await search_routes_with_details(route_service, query_text=query, limit=limit)
    if results:
        route_certainties = {}
        weight = 1.0
        total_weight = 0.0

        for res in results:
            route_name = res["route"]
            certainty = res["certainty"]
            weighted_certainty = certainty * weight
            route_certainties[route_name] = route_certainties.get(route_name, 0) + weighted_certainty

            total_weight += weight
            weight *= weight_decay  # Decrease weight for next rank

        # Normalize certainties
        for route_name in route_certainties:
            route_certainties[route_name] /= total_weight

        # Determine the route with the highest weighted certainty
        chosen_route = max(route_certainties, key=route_certainties.get)
        return chosen_route
    else:
        return "no_route"


#### Run Evaluation for Both Methods


In [12]:
final_test = [
    # clear intent
    {
        "query": "What are the top Computer on Modules available with high memory and I/O count?",
        "route": "clear_intent_product",
    },
    {
        "query": "Can you list some single board computers suitable for industrial-grade applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "List single board computers that offer extensive graphical processing capabilities.",
        "route": "clear_intent_product",
    },
    {
        "query": "What development kits are ideal for rapid prototyping of IoT applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which single board computers come with long-term support and are highly customizable?",
        "route": "clear_intent_product",
    },
    {
        "query": "Find Computer on Modules that can operate in extended temperature ranges.",
        "route": "clear_intent_product",
    },
    {
        "query": "Which SBCs provide the best price-to-performance ratio for educational purposes?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best development kits for working with FPGA and high-speed data transceivers?",
        "route": "clear_intent_product",
    },
    {"query": "What single board computers offer native support for Android OS?", "route": "clear_intent_product"},
    {
        "query": "What are the best SBCs for running IoT applications with Bluetooth support?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which development kits are best for evaluating high-efficiency step-down regulators?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the top single board computers for industrial applications with extended temperature range?",
        "route": "clear_intent_product",
    },
    {
        "query": "What Computer on Modules (COMs) can be recommended for advanced graphics applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which Single Board Computers (SBCs) are suited for AI applications with onboard AI accelerator?",
        "route": "clear_intent_product",
    },
    {
        "query": "What DevKits offer comprehensive prototyping capabilities for IoT edge applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which SBCs are best suited for home automation systems considering the size and power consumption?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best SBCs for robotics applications with multiple GPIO and PWM support?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which embedded boards are best for real-time data processing applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "What COMs are recommended for high-performance computing in harsh environments?",
        "route": "clear_intent_product",
    },
    {"query": "Which SBCs are best suited for educational and DIY projects?", "route": "clear_intent_product"},
    {
        "query": "What development kits provide support for LoRa and other low-power WAN technologies?",
        "route": "clear_intent_product",
    },
    {"query": "Which Computer on Modules (COMs) are suitable for healthcare devices?", "route": "clear_intent_product"},
    {"query": "What SBCs can be recommended for AI-powered security camera systems?", "route": "clear_intent_product"},
    {
        "query": "Which DevKits are best suited for learning wireless communications and IoT?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best Computer on Modules (COMs) for autonomous vehicle applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "What are the best Computer on Modules (COMs) for autonomous vehicle applications?",
        "route": "clear_intent_product",
    },
    {
        "query": "Which single board computers offer the highest memory capacity and what is their size/form factor and manufacturer?",
        "route": "clear_intent_product",
    },
    {
        "query": "Could you provide single board computers that are compatible with WiFi modules?",
        "route": "clear_intent_product",
    },
    {
        "query": "Can you list some Development Kits that are suitable for evaluating high-efficiency step-down regulators?",
        "route": "clear_intent_product",
    },
    {
        "query": "What development kits support multiple phases for power supply design?",
        "route": "clear_intent_product",
    },
    # vague intent
    {"query": "Provide a list of products with ARM Cortex processors.", "route": "vague_intent_product"},
    {"query": "Tell me about the latest trends in computer technology.", "route": "vague_intent_product"},
    {"query": "What are the key features of the new MacBook Pro?", "route": "vague_intent_product"},
    {"query": "Can you recommend a good book on computer programming?", "route": "vague_intent_product"},
    {"query": "What are the main advantages of using AI in business?", "route": "vague_intent_product"},
    {"query": "How does cloud computing impact the IT industry?", "route": "vague_intent_product"},
    {"query": "What are the latest advancements in AI technology?", "route": "vague_intent_product"},
    {"query": "Can you explain how blockchain technology works?", "route": "vague_intent_product"},
    {"query": "What are the benefits of using renewable energy sources?", "route": "vague_intent_product"},
    {"query": "How does AI affect the future of work?", "route": "vague_intent_product"},
    # chitchat
    {"query": "Hi there! How are you?", "route": "chitchat"},
    {"query": "Tell me a joke!", "route": "chitchat"},
    {"query": "How's the weather today?", "route": "chitchat"},
    {"query": "What's your name?", "route": "chitchat"},
    {"query": "Can you tell me a secret?", "route": "chitchat"},
    {"query": "What's your favorite color?", "route": "chitchat"},
    {"query": "Can you sing a song?", "route": "chitchat"},
    {"query": "What's your favorite food?", "route": "chitchat"},
    # politics
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
    {"query": "What's the latest news on the elections?", "route": "politics"},
    {"query": "Let's discuss government policies.", "route": "politics"},
]

In [13]:
print("=== Evaluating Method 1: Certainty Threshold ===")
await evaluate_method(
    route_service,
    final_test,
    routing_with_certainty_threshold_eval,
    certainty_threshold=0.75,
)


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"


=== Evaluating Method 1: Certainty Threshold ===


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Results:
Total Queries: 56
Correct Predictions: 12
Accuracy: 21.43%

Misclassifications:
clear_intent_product -> uncertain: 24
clear_intent_product -> vague_intent_product: 5
vague_intent_product -> uncertain: 10
chitchat -> uncertain: 5


In [14]:

print("\n=== Evaluating Method 2: Weighted Certainty Aggregation ===")
await evaluate_method(
    route_service,
    final_test,
    weighted_aggregation_routing_eval,
    limit=5,
)


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



=== Evaluating Method 2: Weighted Certainty Aggregation ===


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Results:
Total Queries: 56
Correct Predictions: 37
Accuracy: 66.07%

Misclassifications:
clear_intent_product -> vague_intent_product: 12
vague_intent_product -> clear_intent_product: 4
vague_intent_product -> politics: 3




---

<a name="conclusion"></a>
## 6. Conclusion



We have implemented two alternative methods to improve the semantic routing system:

1. **Certainty Threshold Method**: By introducing a certainty threshold, the system can now identify low-confidence classifications and handle them appropriately.

2. **Weighted Certainty Aggregation**: This method improves upon simple aggregation by weighting the certainties based on their rank, giving more influence to higher-ranked results.
